原文在这里：

https://www.dataapplab.com/introduction-time-series-classification/

关于写这篇文章的目的，是因为最近包括之前遇到了中分组时间序列数据，目标是：对每个对象进行分类 数据：每个对象都有多条数据，是时间序列的形式。

总结在下深度学习的做法

## 简介

我们大多数人所接触的时间序列数据主要用于生成预测。无论是预测产品的需求或销售，航空公司的乘客数量，还是某只股票的收盘价，我们都习惯于利用成熟的时间序列技术来预测需求。

但是，随着生成的数据量呈指数级增长，尝试新思想和新算法的机会也随之增加。处理复杂的时间序列数据集仍然是一个有潜力的领域，扩展您的库以包含新想法总是有帮助的。

我在这篇文章中要做的就是向你们介绍时间序列分类的创新概念。我们将首先了解这个主题的含义和它在行业中的应用。但是，我们不会停留在理论部分—我们将着手处理时间序列数据集并执行二进制时间序列分类。边做边学——这也会帮助你以实践的方式理解这个概念。

如果你以前没有研究过时间序列问题，我强烈建议你先从一些基本的预测开始。你可以先看看这篇文章: A comprehensive beginner’s guide to create a Time Series Forecast (with Codes in Python)。

## 目录

1. 时间序列分类简介
    a. ECG信号
    b. 图像数据
    c. 传感器
2. 问题描述
3. 读取和理解数据
4. 预先处理
5. 建立时间序列分类模型

## 时间序列分类简介

 时间序列分类实际上已经存在一段时间了。但到目前为止，它主要局限于实验室研究，而不是工业应用。不过由于有很多研究正在进行，新的数据集正在创建，一些新的算法也正在提出。当我第一次遇到时间序列分类的概念时，我最初的想法是：我们如何对时间序列进行分类？时间序列分类的数据是什么样子的？

可以想象，时间序列分类数据不同于常规分类问题，因为属性具有有序的序列。让我们来看看一些时间序列分类用例来理解这种差异。

1) ECG/EEG信号分类

心电图（ECG，Electrocardiogram）记录着心脏的电活动，被广泛地用于诊断各种心脏问题。这些心电信号是用外部电极捕捉的。例如，考虑下面的信号样本，它表示一个心跳的电活动。左边的图像表示正常的心跳，而相邻的图像表示心肌梗死。
![](https://www.dataapplab.com/wp-content/uploads/2019/07/%E5%BE%AE%E4%BF%A1%E5%9B%BE%E7%89%87_20190722163302.jpg)

2) 图像数据

图像也可以是时间序列相关的格式。考虑以下场景:

作物生长在特定的领域取决于天气条件、土壤肥力、水的可用性和其他外部因素。这片土地的照片是连续5年每天拍摄的，并标有种植在这片土地上的作物的名称。数据集中的图像是在固定的时间间隔后拍摄的，并且有一个确定的序列，这是对图像进行分类的一个重要因素。

3) 动作传感器数据分类

传感器产生高频数据，可以识别出物体在其范围内的运动。通过设置多个无线传感器，观察传感器信号强度的变化，可以识别出物体的运动方向。

你还能想到什么其他的应用我们可以应用时间序列分类？请在文章下面的评论部分告诉我们。

问题描述

下面我们以“室内用户运动预测”这个问题为例。在这个挑战中，多个运动传感器被放置在不同的房间中，目标是根据从这些运动传感器捕捉到的频率数据来识别一个人是否在房间中移动过。

一共有四个运动传感器(A1、A2、A3、A4)分布在两个房间。请看下图，它说明了传感器在每个房间的位置。这两个房间的设置是在3对不同的房间组中创建的(group1、group2、group3)。

一个人可以沿着上图所示的六个预定义路径中的任意一条移动。如果一个人走在2号、3号、4号或6号路上，他就会在房间里移动。另一方面，如果一个人遵循路径1或路径5，我们可以说这个人已经在房间之间移动了。

传感器读数可以用来识别一个人在给定时间点的位置。当人在房间里走动或穿越房间时，传感器中的读数会发生变化。此更改可用于标识人员的路径。

现在问题陈述已经很清楚了，是时候开始编写代码了!在下一节中，我们将查看问题的数据集，它将帮助您清除关于此语句的任何遗留问题。您可以从这个链接下载数据集:https://archive.ics.uci.edu/ml/datasets/Indoor+User+Movement+Prediction+from+RSS+data.

## 读取和理解数据

我们的数据集包括316个文件:

     · 314个MovementAAL 数据集，包含环境中运动传感器的读数
    · 包含每个MovementAAL文件的目标变量的Target 数据集
    · 一个Group数据集来标识不同的移动文件属于哪个设置组
    · 包含对象所取路径的路径数据文件

让我们看一下数据集。我们将从导入对应的数据库开始。

Python Code：

In [1]:
import pandas as pd
import numpy as np
%matplotlib inline

In [2]:
# import matplotlib.pyplot as plt
# from os import listdir
# import tensorflow.keras as keras
# from tensorflow.keras.preprocessing import sequence
# import tensorflow as tf
# from tensorflow.keras.models import Sequential,Model
# from tensorflow.keras.layers import *
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.models import load_model

# from tensorflow.keras.callbacks import ModelCheckpoint

In [24]:
import matplotlib.pyplot as plt
from os import listdir
import keras
from keras.preprocessing import sequence
import tensorflow as tf
from keras.models import Sequential,Model
from keras.layers import *
from keras.optimizers import Adam
from keras.models import load_model
from keras.callbacks import ModelCheckpoint

在载入数据之前，我们一个快速看一下我们要处理的数据。读取移动数据的前两个文件：

In [4]:
df1=pd.read_csv('data/MovementAAL/dataset/MovementAAL_RSS_1.csv')
df2=pd.read_csv('data/MovementAAL/dataset/MovementAAL_RSS_2.csv')

In [5]:
df1.head()

,#RSS_anchor1,RSS_anchor2,RSS_anchor3,RSS_anchor4
0,-0.90476,-0.48,0.28571,0.30
1,-0.57143,-0.32,0.14286,0.30
2,-0.38095,-0.28,-0.14286,0.35
3,-0.28571,-0.20,-0.47619,0.35
4,-0.14286,-0.20,0.14286,-0.20


In [6]:
df2.head()

,#RSS_anchor1,RSS_anchor2,RSS_anchor3,RSS_anchor4
0,-0.57143,-0.20,0.71429,0.50
1,-0.76190,-0.48,0.76190,-0.25
2,-0.85714,-0.60,0.85714,0.55
3,-0.76190,-0.40,0.71429,0.60
4,-0.76190,-0.84,0.85714,0.45


In [7]:
df1.shape

(27, 4)

这些文件包含来自四个传感器(A1、A2、A3、A4)的标准化数据。csv文件的长度(行数)不同，因为每个csv对应的数据的持续时间不同。为了简化问题，让我们假设每秒钟收集一次传感器数据。第一次读取持续时间为27秒(即27行)，而另一次读取持续时间为26秒(即26行)。

在建立模型之前，我们必须处理这个变化的长度。现在，我们将使用以下代码块从传感器中读取并存储列表的值:

In [8]:
path='data/MovementAAL/dataset/MovementAAL_RSS_'
sequences=list()
for i in range(1,315):
    file_path=path+str(i)+'.csv'
    print(file_path)
    df=pd.read_csv(file_path,header=0)
    values=df.values
    sequences.append(values)
targets=pd.read_csv('data/MovementAAL/dataset/MovementAAL_target.csv')
targets=targets.values[:,1]

data/MovementAAL/dataset/MovementAAL_RSS_1.csv
data/MovementAAL/dataset/MovementAAL_RSS_2.csv
data/MovementAAL/dataset/MovementAAL_RSS_3.csv
data/MovementAAL/dataset/MovementAAL_RSS_4.csv
data/MovementAAL/dataset/MovementAAL_RSS_5.csv
data/MovementAAL/dataset/MovementAAL_RSS_6.csv
data/MovementAAL/dataset/MovementAAL_RSS_7.csv
data/MovementAAL/dataset/MovementAAL_RSS_8.csv
data/MovementAAL/dataset/MovementAAL_RSS_9.csv
data/MovementAAL/dataset/MovementAAL_RSS_10.csv
data/MovementAAL/dataset/MovementAAL_RSS_11.csv
data/MovementAAL/dataset/MovementAAL_RSS_12.csv
data/MovementAAL/dataset/MovementAAL_RSS_13.csv
data/MovementAAL/dataset/MovementAAL_RSS_14.csv
data/MovementAAL/dataset/MovementAAL_RSS_15.csv
data/MovementAAL/dataset/MovementAAL_RSS_16.csv
data/MovementAAL/dataset/MovementAAL_RSS_17.csv
data/MovementAAL/dataset/MovementAAL_RSS_18.csv
data/MovementAAL/dataset/MovementAAL_RSS_19.csv
data/MovementAAL/dataset/MovementAAL_RSS_20.csv
data/MovementAAL/dataset/MovementAAL_RSS_21.csv
d

data/MovementAAL/dataset/MovementAAL_RSS_297.csv
data/MovementAAL/dataset/MovementAAL_RSS_298.csv
data/MovementAAL/dataset/MovementAAL_RSS_299.csv
data/MovementAAL/dataset/MovementAAL_RSS_300.csv
data/MovementAAL/dataset/MovementAAL_RSS_301.csv
data/MovementAAL/dataset/MovementAAL_RSS_302.csv
data/MovementAAL/dataset/MovementAAL_RSS_303.csv
data/MovementAAL/dataset/MovementAAL_RSS_304.csv
data/MovementAAL/dataset/MovementAAL_RSS_305.csv
data/MovementAAL/dataset/MovementAAL_RSS_306.csv
data/MovementAAL/dataset/MovementAAL_RSS_307.csv
data/MovementAAL/dataset/MovementAAL_RSS_308.csv
data/MovementAAL/dataset/MovementAAL_RSS_309.csv
data/MovementAAL/dataset/MovementAAL_RSS_310.csv
data/MovementAAL/dataset/MovementAAL_RSS_311.csv
data/MovementAAL/dataset/MovementAAL_RSS_312.csv
data/MovementAAL/dataset/MovementAAL_RSS_313.csv
data/MovementAAL/dataset/MovementAAL_RSS_314.csv


In [9]:
sequences[0]

array([[-0.90476 , -0.48    ,  0.28571 ,  0.3     ],
       [-0.57143 , -0.32    ,  0.14286 ,  0.3     ],
       [-0.38095 , -0.28    , -0.14286 ,  0.35    ],
       [-0.28571 , -0.2     , -0.47619 ,  0.35    ],
       [-0.14286 , -0.2     ,  0.14286 , -0.2     ],
       [-0.14286 , -0.2     ,  0.047619,  0.      ],
       [-0.14286 , -0.16    , -0.38095 ,  0.2     ],
       [-0.14286 , -0.04    , -0.61905 , -0.2     ],
       [-0.095238, -0.08    ,  0.14286 , -0.55    ],
       [-0.047619,  0.04    , -0.095238,  0.05    ],
       [-0.19048 , -0.04    ,  0.095238,  0.4     ],
       [-0.095238, -0.04    , -0.14286 ,  0.35    ],
       [-0.33333 , -0.08    , -0.28571 , -0.2     ],
       [-0.2381  ,  0.04    ,  0.14286 ,  0.35    ],
       [ 0.      ,  0.08    ,  0.14286 ,  0.05    ],
       [-0.095238,  0.04    ,  0.095238,  0.1     ],
       [-0.14286 , -0.2     ,  0.14286 ,  0.5     ],
       [-0.19048 ,  0.04    , -0.42857 ,  0.3     ],
       [-0.14286 , -0.08    , -0.2381  ,  0.15

In [10]:
targets

array([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1

如前所述，数据集是在三对不同的房间组中收集的，因此分为三组。此信息可用于将数据集划分为训练集、测试集和验证集。我们现在加载数据组csv文件:

In [11]:
groups=pd.read_csv('data/MovementAAL/groups/MovementAAL_DatasetGroup.csv',header=0)
groups=groups.values[:,1]

我们将取前两组数据用于训练集，取第三组数据用于测试集。

## 预先处理

由于时间序列数据的长度是变化的，我们不能直接在这个数据集上建立模型。那么如何确定一个级数的理想长度呢？我们可以通过多种方式来解决这个问题，这里有一些想法(我很乐意在评论部分听到你的建议):

- 用零填充较短的序列，使所有序列的长度相等。但在这种情况下，我们将向模型提供一些不正确的数据
- 找出序列的最大长度，并用最后一行中的数据填充序列
- 在数据集中确定序列的最小长度，并将所有其他序列截断到该长度。然而，这将导致数据的巨大损失
- 取所有长度的平均值，截断较长的序列，填充比平均长度短的序列求出最小、最大和平均长度

我们来找出最小、最大和平均长度：

In [12]:
len_sequences=[]
for one_seq in sequences:
    len_sequences.append(len(one_seq))
pd.Series(len_sequences).describe()

count    314.000000
mean      42.028662
std       16.185303
min       19.000000
25%       26.000000
50%       41.000000
75%       56.000000
max      129.000000
dtype: float64

大多数文件的长度在40到60之间。只有3个文件的长度超过100。因此，取最小或最大长度没有多大意义。第90个四分位数是60，我们把这个作为数据的序列长度。让我们把它码出来:

In [13]:
# padding the sequence with the values in last row to max length
to_pad=129
new_seq=[]
for one_seq in sequences:
    len_one_seq=len(one_seq)
    last_val=one_seq[-1]
    n=to_pad-len_one_seq
    to_concat=np.repeat(last_val,n).reshape(4,n).transpose()
    new_one_seq=np.concatenate([one_seq,to_concat])
    new_seq.append(new_one_seq)

final_seq=np.stack(new_seq)
# final_seq.shape (314, 129, 4)
final_seq.shape
# 进行截断


(314, 129, 4)

In [14]:
from keras.preprocessing import sequence

In [15]:
seq_len=60
final_seq=sequence.pad_sequences(final_seq,maxlen=seq_len,padding='post',
                                dtype='float',truncating='post')
final_seq.shape

(314, 60, 4)

现在我们的数据集已经准备好，我们将会根据Group将它分开。准备好训练集，验证集和测试集：

In [16]:
train=[final_seq[i] for i in range(len(groups)) if(groups[i])==2]
validation=[final_seq[i] for i in range(len(groups)) if(groups[i])==1]
test=[final_seq[i] for i in range(len(groups)) if(groups[i])==3]



train_target=[targets[i] for i in range(len(groups)) if(groups[i])==2]
validation_target=[targets[i] for i in range(len(groups)) if(groups[i])==1]
test_target=[targets[i] for i in range(len(groups)) if(groups[i])==3]

train=np.array(train)
validation=np.array(validation)
test=np.array(test)

print("train.shape:",train.shape)
print("validation.shape:",validation.shape)
print("test.num:",test.shape)

train_target=np.array(train_target)
validation_target=np.array(validation_target)
test_target=np.array(test_target)

train_target=(train_target+1)/2
validation_target=(validation_target+1)/2
test_target=(test_target+1)/2
test_target

train.shape: (106, 60, 4)
validation.shape: (104, 60, 4)
test.num: (104, 60, 4)


array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.])

## 建立时间序列分类模型

我们已经准备好了用于LSTM(长短期内存)模型的数据。我们处理可变长度序列并创建了训练、验证和测试集。让我们构建一个单层LSTM网络。

注意:您可以在本教程中熟悉LSTMs。我建议您首先完成这些操作，因为这将帮助您理解下面的代码是如何工作的。

In [17]:
# model=Sequential()
# model.add(LSTM(256,input_shape=(seq_len,4)))
# model.add(Dense(1,activation='sigmoid'))
# model.summary()

In [18]:
input_layer=Input(shape=(seq_len,4),name="input_layer")
lstm_layer=LSTM(256)(input_layer)
pred=Dense(1,activation='sigmoid')(lstm_layer)
model=Model(input_layer,pred)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     [(None, 60, 4)]           0         
_________________________________________________________________
lstm (LSTM)                  (None, 256)               267264    
_________________________________________________________________
dense (Dense)                (None, 1)                 257       
Total params: 267,521
Trainable params: 267,521
Non-trainable params: 0
_________________________________________________________________


In [19]:
adam=Adam(lr=0.001)
checkpoint=ModelCheckpoint('data/best_model.pkl',monitor='accuracy',save_best_only=True,verbose=1)
model.compile(loss="binary_crossentropy",optimizer=adam,metrics=['accuracy'])

In [20]:
model.fit(
    train,
    train_target,
    epochs=100,
    batch_size=32,
    callbacks=[checkpoint],
    validation_data=(validation,validation_target)
)

Epoch 1/100
1/4 [======>.......................] - ETA: 0s - loss: 0.6999 - accuracy: 0.4375
Epoch 00001: accuracy improved from -inf to 0.48113, saving model to data/best_model.pkl
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: data/best_model.pkl\assets
4/4 [==============================] - 3s 764ms/step - loss: 0.7022 - accuracy: 0.4811 - val_loss: 0.6712 - val_accuracy: 0.5962
Epoch 2/100
1/4 [======>.......................] - ETA: 0s - loss: 0.6753 - accuracy: 0.6562
Epoch 00002: accuracy improved from 0.48113 to 0.68868, saving model to data/best_model.pkl
INFO:tensorflow:Assets written to: data/best_model.pkl\assets
4/4 [==============================] - 3s 684ms/step - loss: 0.6554 - accuracy: 0.6887 - val_loss: 0.6552 - val_accuracy: 0.5385
Epoch 3/100
1/4 [======>.......................] - ETA: 0s - loss: 0.6288 - accuracy: 0.6250
Epoch 00003: accuracy did not improve from 0.68868
4/4 [=====================

Epoch 26/100
1/4 [======>.......................] - ETA: 0s - loss: 0.3866 - accuracy: 0.8438
Epoch 00026: accuracy did not improve from 0.91509
4/4 [==============================] - 0s 13ms/step - loss: 0.2979 - accuracy: 0.9057 - val_loss: 1.2464 - val_accuracy: 0.5673
Epoch 27/100
1/4 [======>.......................] - ETA: 0s - loss: 0.1403 - accuracy: 1.0000
Epoch 00027: accuracy did not improve from 0.91509
4/4 [==============================] - 0s 13ms/step - loss: 0.2684 - accuracy: 0.9151 - val_loss: 0.7787 - val_accuracy: 0.7115
Epoch 28/100
1/4 [======>.......................] - ETA: 0s - loss: 0.4502 - accuracy: 0.8125
Epoch 00028: accuracy did not improve from 0.91509
4/4 [==============================] - 0s 13ms/step - loss: 0.2700 - accuracy: 0.8868 - val_loss: 1.5923 - val_accuracy: 0.5865
Epoch 29/100
1/4 [======>.......................] - ETA: 0s - loss: 0.2304 - accuracy: 0.8750
Epoch 00029: accuracy improved from 0.91509 to 0.93396, saving model to data/best_model

Epoch 56/100
1/4 [======>.......................] - ETA: 0s - loss: 0.2548 - accuracy: 0.9062
Epoch 00056: accuracy did not improve from 0.93396
4/4 [==============================] - 0s 14ms/step - loss: 0.2927 - accuracy: 0.8774 - val_loss: 1.6250 - val_accuracy: 0.6346
Epoch 57/100
1/4 [======>.......................] - ETA: 0s - loss: 0.3246 - accuracy: 0.8750
Epoch 00057: accuracy did not improve from 0.93396
4/4 [==============================] - 0s 12ms/step - loss: 0.3202 - accuracy: 0.8679 - val_loss: 1.7557 - val_accuracy: 0.6058
Epoch 58/100
1/4 [======>.......................] - ETA: 0s - loss: 0.2367 - accuracy: 0.9688
Epoch 00058: accuracy did not improve from 0.93396
4/4 [==============================] - 0s 13ms/step - loss: 0.3461 - accuracy: 0.8774 - val_loss: 1.7536 - val_accuracy: 0.6635
Epoch 59/100
1/4 [======>.......................] - ETA: 0s - loss: 0.3716 - accuracy: 0.9062
Epoch 00059: accuracy did not improve from 0.93396
4/4 [==============================]

1/4 [======>.......................] - ETA: 0s - loss: 0.2288 - accuracy: 0.9375
Epoch 00086: accuracy did not improve from 0.93396
4/4 [==============================] - 0s 13ms/step - loss: 0.2331 - accuracy: 0.8962 - val_loss: 0.9915 - val_accuracy: 0.6058
Epoch 87/100
1/4 [======>.......................] - ETA: 0s - loss: 0.2556 - accuracy: 0.8750
Epoch 00087: accuracy did not improve from 0.93396
4/4 [==============================] - 0s 13ms/step - loss: 0.2061 - accuracy: 0.8962 - val_loss: 1.0828 - val_accuracy: 0.6346
Epoch 88/100
1/4 [======>.......................] - ETA: 0s - loss: 0.1987 - accuracy: 0.8750
Epoch 00088: accuracy did not improve from 0.93396
4/4 [==============================] - 0s 13ms/step - loss: 0.1785 - accuracy: 0.8868 - val_loss: 1.3654 - val_accuracy: 0.6442
Epoch 89/100
1/4 [======>.......................] - ETA: 0s - loss: 0.3463 - accuracy: 0.8438
Epoch 00089: accuracy did not improve from 0.93396
4/4 [==============================] - 0s 12ms/st

In [75]:

# 加载以前保存的权重
model=tf.keras.models.load_model('./data/best_model.pkl')
from sklearn.metrics import accuracy_score
test_preds=model.predict(test)
test_preds = (test_preds >= 0.5).astype(np.int)
accuracy_score(test_target,test_preds)

0.75